### Запуск библиотек, подключение к Google Drive

In [1]:
!pip install pandas nltk
!pip install pymorphy3
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 660.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
#Для работы на TPU
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.0
    Uninstalling te

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pymorphy2
import random
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
import hashlib
import nlpaug.augmenter.word as naw

# Загрузка необходимых ресурсов для токенизации на русском языке
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Создание словаря услуг

In [ ]:
spr = pd.read_excel('/content/drive/MyDrive/HaK_MEDCI/train/spravochnik.xlsx')
spr

,service_id,ServiceCode,ServiceName
0,0,F01.01.01.1.001,"Прием (осмотр, консультация) врача-акушера-гин..."
1,1,F05.01.01.1.001,"Прием (осмотр, консультация) врача-гастроэнтер..."
2,2,F14.01.01.1.002,"Прием (осмотр, консультация) врача-кардиолога ..."
3,3,F31.05.01.1.001,Взятие крови из периферической вены
4,4,F32.01.01.1.001,"Прием (осмотр, консультация) врача-оторинолари..."
...,...,...,...
70,70,L18.41.02.0.001,Антистрептолизин-О (АСЛ-О)
71,71,L18.41.02.0.002,Ревматоидный фактор
72,72,L18.41.02.0.003,"С-реактивный белок (СРБ, CRP)"
73,73,L18.46.01.0.088,"Коронавирус COVID-19( SARS CoV-2), определение..."


In [ ]:
no_service = pd.DataFrame(columns=spr.columns)
no_service.at[1, 'service_id'] = 75
no_service.at[1, 'ServiceCode'] = 'None'
no_service.at[1, 'ServiceName'] = 'Нет услуг'

In [ ]:
spr_new = spr.append(no_service, ignore_index=True)

<ipython-input-5-9d4ae3e60179>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  spr_new = spr.append(no_service, ignore_index=True)


In [ ]:
spr_new

,service_id,ServiceCode,ServiceName
0,0,F01.01.01.1.001,"Прием (осмотр, консультация) врача-акушера-гин..."
1,1,F05.01.01.1.001,"Прием (осмотр, консультация) врача-гастроэнтер..."
2,2,F14.01.01.1.002,"Прием (осмотр, консультация) врача-кардиолога ..."
3,3,F31.05.01.1.001,Взятие крови из периферической вены
4,4,F32.01.01.1.001,"Прием (осмотр, консультация) врача-оторинолари..."
...,...,...,...
71,71,L18.41.02.0.002,Ревматоидный фактор
72,72,L18.41.02.0.003,"С-реактивный белок (СРБ, CRP)"
73,73,L18.46.01.0.088,"Коронавирус COVID-19( SARS CoV-2), определение..."
74,74,L18.50.00.0.021,Госпитал. хирург. (Кл. анализ крови; СОЭ; Гр.к...


In [ ]:
spr_new.set_index('service_id', inplace=True)

# Преобразуем датафрейм в словарь
dictionary = spr_new.to_dict('index')


In [ ]:
dictionary

{0: {'ServiceCode': 'F01.01.01.1.001',
  'ServiceName': 'Прием (осмотр, консультация) врача-акушера-гинеколога первичный'},
 1: {'ServiceCode': 'F05.01.01.1.001',
  'ServiceName': 'Прием (осмотр, консультация) врача-гастроэнтеролога первичный'},
 2: {'ServiceCode': 'F14.01.01.1.002',
  'ServiceName': 'Прием (осмотр, консультация) врача-кардиолога повторный'},
 3: {'ServiceCode': 'F31.05.01.1.001',
  'ServiceName': 'Взятие крови из периферической вены'},
 4: {'ServiceCode': 'F32.01.01.1.001',
  'ServiceName': 'Прием (осмотр, консультация) врача-оториноларинголога первичный'},
 5: {'ServiceCode': 'F48.01.00.1.001',
  'ServiceName': 'Прием (осмотр, консультация) врача-стоматолога первичный'},
 6: {'ServiceCode': 'F48.08.01.1.003',
  'ServiceName': 'Инфильтрационная анестезия (в стоматологии)'},
 7: {'ServiceCode': 'F48.56.01.1.006',
  'ServiceName': 'Снятие пигментированного налета (система Air-Flow, 1 зуб)'},
 8: {'ServiceCode': 'F48.56.01.1.012',
  'ServiceName': 'Гигиена полости рта и 

In [ ]:
dictionary.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75])

### Создаем обработанный train

In [ ]:
train_org = pd.read_excel('/content/drive/MyDrive/HaK_MEDCI/train/train.xlsx')
train_org.head()

,letter_id,guarantee_letter_text,service_ids_list,class
0,30624374,Ингосстрах \n \nF70.04.02.0.001 Магнитно-резон...,"[32, 18]",1
1,30637539,Согаз \n \nОрганизовать приемы врача (ревматол...,NaN,0
2,30546372,Согаз \n \nОрганизовать приемы врача терапе...,NaN,0
3,30718879,СОГАЗ \n \nОрганизовать приемы врача офталь...,NaN,0
4,30597751,"АО ""АльфаСтрахование"" г.Москва благодарит з...","[34, 39]",1


In [ ]:
train_copy = train_org.copy()

In [ ]:
# Замена NaN на 75 в столбце service_ids_list
train_copy['service_ids_list'] = train_copy['service_ids_list'].fillna('[75]')
train_copy.head()

,letter_id,guarantee_letter_text,service_ids_list,class
0,30624374,Ингосстрах \n \nF70.04.02.0.001 Магнитно-резон...,"[32, 18]",1
1,30637539,Согаз \n \nОрганизовать приемы врача (ревматол...,[75],0
2,30546372,Согаз \n \nОрганизовать приемы врача терапе...,[75],0
3,30718879,СОГАЗ \n \nОрганизовать приемы врача офталь...,[75],0
4,30597751,"АО ""АльфаСтрахование"" г.Москва благодарит з...","[34, 39]",1


In [ ]:
#Датасет вручную нами переработанный, где 1 строчке - 1 услуга
uslugi_orig = pd.read_excel('/content/drive/MyDrive/HaK_MEDCI/train/uslugi.xlsx')
uslugi_copy = uslugi_orig.copy()
#uslugi_orig['letter_id'] = uslugi_orig['letter_id'].np.random.randint(100000,99904099)

In [ ]:
uslugi_orig['service_ids_list'] = uslugi_orig['service_ids_list'].str.replace(r"[^\d\.]", "", regex=True)

In [ ]:
# Определение функции для объединения строк
def merge_rows(df):
    #Объединяем письма - для расширения датасета
    dataframe_2 = pd.DataFrame(columns=df.columns)
    for i in range(len(df) - 1):
        # Выбор строк
        row_1 = df.iloc[i]
        row_2 = df.iloc[i + 1]

        # Создание нового letter_id (простой способ)
        new_letter_id = np.random.randint(10000000, 99999999)
        while new_letter_id in df['letter_id']:
            new_letter_id = new_letter_id

        # Объединение текстов писем
        combined_text = row_1['guarantee_letter_text'] + "." + row_2['guarantee_letter_text']

        # Объединение списков индентификаторов услуг
        #df['service_ids_list'] = df['service_ids_list'].astype
        combined_service_list = list(set(row_1['service_ids_list'] + row_2['service_ids_list']))
                # Определение класса
        combined_class = 1 if combined_service_list else 0

        dataframe_2 = dataframe_2.append({
            'letter_id': new_letter_id,
            'guarantee_letter_text': combined_text,
            'service_ids_list': combined_service_list,
            'class': combined_class
        }, ignore_index=True)

    return dataframe_2

    return new_df

In [ ]:
df_new = merge_rows(uslugi_orig)

In [ ]:
df_Sveta_mixed = pd.concat([uslugi_orig, df_new], ignore_index = True)
df_Sveta_mixed

,letter_id,guarantee_letter_text,service_ids_list,class
0,30906460,"Страховая Компания ПАО ""Группа Ренессанс Ст...",0,1
1,30906460,"Страховая Компания ПАО ""Группа Ренессанс Ст...",1,1
2,30858383,"АО ""СОГАЗ"" \n %СОГЛ% \nПрименение фторсодержащ...",10,1
3,30907300,"САО ""РЕСО-Гарантия"" гарантирует оплату пере...",11,1
4,30777882,"Страховая Компания ПАО ""Группа Ренессанс Ст...",12,1
...,...,...,...,...
160,96291009,"САО ""РЕСО-Гарантия"" гарантирует оплату пере...","[7, 2, 1]",1
161,59367213,"САО ""РЕСО-Гарантия"" гарантирует оплату пере...","[7, 2, 3]",1
162,97612840,Ингосстрах \n \nL18.46.01.0.088 Коронавирус CO...,"[7, 3, 4]",1
163,11804697,АО «СОГАЗ» \n \nПредоперационная подготовка: Г...,"[7, 8, 4]",1


In [ ]:
# Убираем знаки - пробелы
def replace_probel(df):
  df = df.replace('\n',' ')
  return df

In [ ]:
#train_ANNA_copy - не добавляем - тестовый
#Соединяем 3 датасета
train_concate = pd.concat([train_copy, df_Sveta_mixed], ignore_index=True)
train_concate['guarantee_letter_text'] = train_concate['guarantee_letter_text'].apply(replace_probel)
train_concate

,letter_id,guarantee_letter_text,service_ids_list,class
0,30624374,Ингосстрах F70.04.02.0.001 Магнитно-резонан...,"[32, 18]",1
1,30637539,Согаз Организовать приемы врача (ревматолог...,[75],0
2,30546372,Согаз Организовать приемы врача терапевт...,[75],0
3,30718879,СОГАЗ Организовать приемы врача офтальмо...,[75],0
4,30597751,"АО ""АльфаСтрахование"" г.Москва благодарит з...","[34, 39]",1
...,...,...,...,...
322,96291009,"САО ""РЕСО-Гарантия"" гарантирует оплату пере...","[7, 2, 1]",1
323,59367213,"САО ""РЕСО-Гарантия"" гарантирует оплату пере...","[7, 2, 3]",1
324,97612840,Ингосстрах L18.46.01.0.088 Коронавирус COVI...,"[7, 3, 4]",1
325,11804697,АО «СОГАЗ» Предоперационная подготовка: Гос...,"[7, 8, 4]",1


### Вытаскиваем ServiceCode

In [ ]:
# Смотрим SetciveCode и вытаскиваем его из основного письма, т.к. при аугментации он нам помешает
ServiceCode = []
for i in dictionary:
    if dictionary[i]['ServiceCode'] not in ServiceCode:
        ServiceCode.append(dictionary[i]['ServiceCode'])
ServiceCode

['F01.01.01.1.001',
 'F05.01.01.1.001',
 'F14.01.01.1.002',
 'F31.05.01.1.001',
 'F32.01.01.1.001',
 'F48.01.00.1.001',
 'F48.08.01.1.003',
 'F48.56.01.1.006',
 'F48.56.01.1.012',
 'F48.56.02.1.003',
 'F48.56.03.1.013',
 'F51.01.01.1.001',
 'F51.01.01.1.002',
 'F55.27.01.1.002',
 'F55.27.02.1.009',
 'F55.27.07.1.007',
 'F55.27.08.1.003',
 'F55.27.08.1.005',
 'F55.27.08.1.023',
 'F56.01.01.1.002',
 'F57.05.01.1.054',
 'F59.09.01.1.001',
 'F59.09.01.1.013',
 'F59.09.01.1.015',
 'F59.09.02.1.001',
 'F64.05.01.0.003',
 'F64.05.01.0.006',
 'F64.05.02.0.005',
 'F69.18.02.0.002',
 'F69.18.02.0.005',
 'F69.20.02.0.004',
 'F69.20.03.0.002',
 'F70.04.02.0.001',
 'F70.04.02.0.007',
 'F70.04.04.0.006',
 'F70.04.05.0.006',
 'F70.04.09.0.001',
 'F70.04.09.0.002',
 'F70.04.09.0.003',
 'F70.04.09.0.004',
 'F70.04.09.0.006',
 'F70.04.09.0.007',
 'F70.04.09.0.008',
 'F73.05.01.1.006',
 'L18.30.00.0.001',
 'L18.30.00.0.005',
 'L18.30.00.0.007',
 'L18.31.01.0.001',
 'L18.31.06.0.001',
 'L18.31.06.0.004',


In [ ]:
def extract_service_code(stroca:str):
    code_list = []
    for code in ServiceCode:
        if stroca.find(code) != -1:
            code_list.append(code)
    #code_str = (', '.join(str(el) for el in code_list))
    return code_list #code_str

In [ ]:
def remove_service_code(stroca:str):
    for code in ServiceCode:
        if stroca.find(code) != -1:
            stroca = stroca.replace(code, '')
    return stroca

In [ ]:
train_concate['ServiceCode'] = train_concate['guarantee_letter_text'].apply(extract_service_code)
train_concate['guarantee_letter_text'] = train_concate['guarantee_letter_text'].apply(remove_service_code)

In [ ]:
train_aug = train_concate.copy()
train_aug.head()

,letter_id,guarantee_letter_text,service_ids_list,class,ServiceCode
0,30624374,Ингосстрах Магнитно-резонансная томография...,"[32, 18]",1,"[F55.27.08.1.023, F70.04.02.0.001]"
1,30637539,Согаз Организовать приемы врача (ревматолог...,[75],0,[]
2,30546372,Согаз Организовать приемы врача терапевт...,[75],0,[]
3,30718879,СОГАЗ Организовать приемы врача офтальмо...,[75],0,[]
4,30597751,"АО ""АльфаСтрахование"" г.Москва благодарит з...","[34, 39]",1,"[F70.04.04.0.006, F70.04.09.0.004]"


### Функции Аугментации

In [ ]:
import nlpaug.augmenter.word.context_word_embs as nawcwe
import nlpaug.augmenter.word.word_embs as nawwe
import nlpaug.augmenter.word.spelling as naws

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# AUGMENT_TEXT
# Функция для аугментации текста - SynonymAug
def Synonym_Aug(text, aug_p=0.3):
    # SyninymAug - здесь можно взять словарь для перефразирований из ppdb или др.
    augmenter = naw.SynonymAug(
        aug_p=aug_p)#, aug_src='ppdb', model_path='/content/drive/MyDrive/HaK_MEDCI/dict/ppdb-1.0-m-phrasal.gz'


    # Аугментация текста
    augmented_text = augmenter.augment(text, n=1)

    return augmented_text

# Количество аугментированных текстов, например, в 2/5/10/100 раз
augmentation_factor = 2

# Создание нового датасета с аугментированными данными
augmented_data = []
for _, row in train_aug.iterrows():
    for _ in range(augmentation_factor):
        augmented_text = Synonym_Aug(row['guarantee_letter_text'])
        augmented_data.append(
            {'letter_id': len(augmented_data),
             'guarantee_letter_text': ' '.join(augmented_text),
             'service_ids_list':row['service_ids_list'],
             'ServiceCode':row['ServiceCode'],
             'class': row['class']})

train_Synonym = pd.DataFrame(augmented_data)
train_Synonym.head()

,letter_id,guarantee_letter_text,service_ids_list,ServiceCode,class
0,0,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
1,1,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
2,2,"Согаз Организовать приемы врача (ревматолог ),...",[75],[],0
3,3,"Согаз Организовать приемы врача (ревматолог ),...",[75],[],0
4,4,"Согаз Организовать приемы врача терапевта, нео...",[75],[],0


In [ ]:
train_Synonym.shape

(1635, 5)

In [ ]:
# AUGMENT_TEXT
# Функция для аугментации текста - RandomWord
def RandomWord_Aug(text, aug_p=0.3):

    augmenter = naw.RandomWordAug(aug_p=aug_p, action = 'substitute')

    # Аугментация текста
    augmented_text = augmenter.augment(text, n=1)

    return augmented_text

# Количество аугментированных текстов, например, в 2/5/10/100 раз
augmentation_factor = 5

# Создание нового датасета с аугментированными данными
augmented_data = []
for _, row in train_Synonym.iterrows():
    for _ in range(augmentation_factor):
        augmented_text = RandomWord_Aug(row['guarantee_letter_text'])
        augmented_data.append(
            {'letter_id': len(augmented_data),
             'guarantee_letter_text': ' '.join(augmented_text),
             'service_ids_list':row['service_ids_list'],
             'ServiceCode':row['ServiceCode'],
             'class': row['class']})

train_Synonym_RandomWord = pd.DataFrame(augmented_data)
train_Synonym_RandomWord.head()

,letter_id,guarantee_letter_text,service_ids_list,ServiceCode,class
0,0,Ингосстрах _ - резонансная томография головног...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
1,1,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
2,2,Ингосстрах Магнитно - _ томография головного м...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
3,3,Ингосстрах Магнитно - резонансная томография _...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
4,4,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1


In [ ]:
train_Synonym_RandomWord.shape

(3270, 5)

In [ ]:
# AUGMENT_TEXT
# Функция для аугментации текста - Spelling
def Spelling_Aug(text, aug_p=0.3):

    augmenter = naw.SpellingAug(aug_p=aug_p)

    # Аугментация текста
    augmented_text = augmenter.augment(text, n=1)

    return augmented_text

# Количество аугментированных текстов, например, в 2/5/10/100 раз
augmentation_factor = 3

# Создание нового датасета с аугментированными данными
augmented_data = []
for _, row in train_Synonym_RandomWord.iterrows():
    for _ in range(augmentation_factor):
        augmented_text = Spelling_Aug(row['guarantee_letter_text'])
        augmented_data.append(
            {'letter_id': len(augmented_data),
             'guarantee_letter_text': ' '.join(augmented_text),
             'service_ids_list':row['service_ids_list'],
             'ServiceCode':row['ServiceCode'],
             'class': row['class']})

train_Synonym_RandomWord_Spelling = pd.DataFrame(augmented_data)
train_Synonym_RandomWord_Spelling.head()

,letter_id,guarantee_letter_text,service_ids_list,ServiceCode,class
0,0,Ингосстрах _ - резонансная томография головног...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
1,1,Ингосстрах _ - резонансная томография головног...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
2,2,Ингосстрах _ - резонансная томография головног...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
3,3,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
4,4,Ингосстрах Магнитно - резонансная томография г...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1


In [ ]:
train_Synonym_RandomWord_Spelling.shape

(9810, 5)

In [ ]:
#import gensim.downloader as api
#word2vec_model = api.load('word2vec-google-news-300')
#import pickle#
#
#file = open('word2vec-model.pkl', 'wb')
#pickle.dump(word2vec_model, file)
#file.close()

In [ ]:
# AUGMENT_TEXT
# Функция для аугментации текста -Antonym
#def WordEmbsAug(text, aug_p=0.3):
#    # WordEmbsAug
#    augmenter = naw.WordEmbsAug(aug_p=0.3, model_type ='word2vec', action='substitute')#

    # Аугментация текста
#    augmented_text = augmenter.augment(text, n=1)

#    return augmented_text

# Количество аугментированных текстов, например, в 2/5/10/100 раз
#augmentation_factor = 10

# Создание нового датасета с аугментированными данными
#augmented_data = []
#for _, row in train_aug.iterrows():
#   for _ in range(augmentation_factor):
#        augmented_text = WordEmbsAug(row['guarantee_letter_text'])
#        augmented_data.append(
#            {'letter_id': len(augmented_data),
#            'guarantee_letter_text': ' '.join(augmented_text),
#             'service_ids_list':row['service_ids_list'],
#             'ServiceCode':row['ServiceCode'],
#             'class': row['class']})

#train_Reserved = pd.DataFrame(augmented_data)
#train_Reserved.head()

In [ ]:
train_Synonym_RandomWord_Spelling['guarantee_letter_text'][0]

'Ингосстрах Магнитно - резонансная томография головного мозга l Дуплексное сканирование экстракраниальных _ брахиоцефальных артерий l на Белорусской Оплата будет осуществляться _ _ с условиями Договора _ _, являющейся приложением к Договору. Оплата медицинских услуг, оказанных по данному гарантийному письму, может быть оспорена по результатам экспертизы счета и / или медицинской документации. Диагноз: Вестибулоатактический синдром, синдром когнитивных _, _, уточнить генез. Исключить гемодинамически значимые стенозы, _, койлинг _. Исключить пароксизмальную активность коры головного мозга. Исключить _ объемное образование. (G93. 5 )'

In [ ]:
train_Synonym_RandomWord_Spelling['guarantee_letter_text'][4]

'_ Магнитно - резонансная _ головного мозга 1 Дуплексное сканирование экстракраниальных отделов брахиоцефальных _ one на Белорусской Оплата _ осуществляться в соответствии с условиями Договора и Программой, являющейся приложением к Договору. Оплата _ услуг, оказанных по данному гарантийному письму, _ быть оспорена по результатам экспертизы счета и / _ медицинской документации. Диагноз: Вестибулоатактический синдром, синдром когнитивных нарушений, _, _ генез. Исключить гемодинамически _ стенозы, кинкинг, койлинг БЦА. Исключить пароксизмальную активность коры головного мозга. Исключить интракраниальное объемное образование. (G93. 5 )'

### Создание датасета

In [ ]:
# Добавляем все датасеты в один
aug_df = pd.concat(
    [train_aug, train_Synonym,
     train_Synonym_RandomWord,
     train_Synonym_RandomWord_Spelling], ignore_index=True)
aug_df

,letter_id,guarantee_letter_text,service_ids_list,class,ServiceCode
0,30624374,Ингосстрах Магнитно-резонансная томография...,"[32, 18]",1,"[F55.27.08.1.023, F70.04.02.0.001]"
1,30637539,Согаз Организовать приемы врача (ревматолог...,[75],0,[]
2,30546372,Согаз Организовать приемы врача терапевт...,[75],0,[]
3,30718879,СОГАЗ Организовать приемы врача офтальмо...,[75],0,[]
4,30597751,"АО ""АльфаСтрахование"" г.Москва благодарит з...","[34, 39]",1,"[F70.04.04.0.006, F70.04.09.0.004]"
...,...,...,...,...,...
14056,9805,"Страховая Компания ПАО "" _ Ренессанс Страхован...","[8, 9]",1,[]
14057,9806,"Страховая Компания ПАО "" _ Ренессанс Страхован...","[8, 9]",1,[]
14058,9807,"Страховая Компания _ "" Группа _ Страхование "" ...","[8, 9]",1,[]
14059,9808,"Страховая Компания _ "" Группа _ Страхование "" ...","[8, 9]",1,[]


In [ ]:
# Перемешиваем предложения.
import nlpaug.augmenter.sentence as nas
# AUGMENT_TEXT
# Функция для аугментации текста - RandomSentAug
def RandomSentAug(text, aug_p=0.3):
    # RandomSentAug
    augmenter = aug = nas.RandomSentAug()

    # Аугментация текста
    augmented_text = augmenter.augment(text, n=1)

    return augmented_text

# Количество аугментированных текстов, например, в 2/5/10/100 раз
augmentation_factor = 3

# Создание нового датасета с аугментированными данными
augmented_data = []
for _, row in aug_df.iterrows():
    for _ in range(augmentation_factor):
        augmented_text = RandomSentAug(row['guarantee_letter_text'])
        augmented_data.append(
            {'letter_id': len(augmented_data),
             'guarantee_letter_text': ' '.join(augmented_text),
             'service_ids_list':row['service_ids_list'],
             'ServiceCode':row['ServiceCode'],
             'class': row['class']})

aug_df_reverseSet = pd.DataFrame(augmented_data)
aug_df_reverseSet.head()

In [ ]:
aug_df_reverseSet

,letter_id,guarantee_letter_text,service_ids_list,ServiceCode,class
0,0,(G93.4) Диагноз: Вестибулоатактический синдром...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
1,1,"Оплата медицинских услуг, оказанных по данному...","[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
2,2,Ингосстрах Магнитно-резонансная томография...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
3,3,"Услуги, оказанные без медицинских показаний, ...",[75],[],0
4,4,"Услуги, оказанные без медицинских показаний, ...",[75],[],0
...,...,...,...,...,...
42178,42178,"2024г. Страховая Компания _ "" Группа _ Страхов...","[8, 9]",[],1
42179,42179,"07. Страховая Компания _ "" Группа _ Страховани...","[8, 9]",[],1
42180,42180,"Страховая Компания _ "" Группа _ Страхование "" ...","[8, 9]",[],1
42181,42181,"Страховая Компания _ "" Группа _ Страхование "" ...","[8, 9]",[],1


In [ ]:
#!pip install pymorphy3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 713.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.1 MB/s eta 0:00:00


In [ ]:
#приводить слово к нормальной форме
#ставить слово в нужную форму. Например, ставить слово во множественное число, менять падеж слова и т.д.
#возвращать грамматическую информацию о слове (число, род, падеж, часть речи и т.д.)

import pymorphy3

#MORHANALYZER
# Инициализация PyMorphy3
morph = pymorphy3.MorphAnalyzer()

# Функция для расширения текста новыми словами на русском языке
def expand_text_with_words(text, num_words_to_add=10):
    # Токенизация текста на слова
    tokens = word_tokenize(text, language='russian')

    # Исключение стоп-слов из русского корпуса
    russian_stopwords = set(stopwords.words('russian'))
    tokens_filtered = [word for word in tokens if word not in russian_stopwords]

    # Лемматизация слов с использованием PyMorphy3
    lemmas = [morph.parse(word)[0].normal_form for word in tokens_filtered]

    # Выбор уникальных слов, не входящих в исходный текст
    new_words = random.sample(lemmas, min(num_words_to_add, len(lemmas)))

    # Добавление новых слов к тексту
    expanded_text = ' '.join(tokens + new_words)

    return expanded_text

In [ ]:
aug_df_reverseSet['guarantee_letter_text'] = aug_df_reverseSet['guarantee_letter_text'].apply(expand_text_with_words)
aug_df_reverseSet

,letter_id,guarantee_letter_text,service_ids_list,ServiceCode,class
0,0,( G93.4 ) Диагноз : Вестибулоатактический синд...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
1,1,"Оплата медицинских услуг , оказанных по данном...","[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
2,2,Ингосстрах Магнитно-резонансная томография гол...,"[32, 18]","[F55.27.08.1.023, F70.04.02.0.001]",1
3,3,"Услуги , оказанные без медицинских показаний ,...",[75],[],0
4,4,"Услуги , оказанные без медицинских показаний ,...",[75],[],0
...,...,...,...,...,...
42178,42178,2024г . Страховая Компания _ `` Группа _ Страх...,"[8, 9]",[],1
42179,42179,07 . Страховая Компания _ `` Группа _ Страхова...,"[8, 9]",[],1
42180,42180,Страховая Компания _ `` Группа _ Страхование `...,"[8, 9]",[],1
42181,42181,Страховая Компания _ `` Группа _ Страхование `...,"[8, 9]",[],1


### Сохранение Датасета!

In [ ]:
# Сохранение нового датасета
aug_df_reverseSet.to_csv('/content/drive/MyDrive/HaK_MEDCI/train/final_aug_df.csv', index=False)

In [ ]:
final_aug_df = pd.read_csv('/content/drive/MyDrive/HaK_MEDCI/train/final_aug_df.csv')
final_aug_df

,letter_id,guarantee_letter_text,service_ids_list,ServiceCode,class
0,0,( G93.4 ) Диагноз : Вестибулоатактический синд...,"[32, 18]","['F55.27.08.1.023', 'F70.04.02.0.001']",1
1,1,"Оплата медицинских услуг , оказанных по данном...","[32, 18]","['F55.27.08.1.023', 'F70.04.02.0.001']",1
2,2,Ингосстрах Магнитно-резонансная томография гол...,"[32, 18]","['F55.27.08.1.023', 'F70.04.02.0.001']",1
3,3,"Услуги , оказанные без медицинских показаний ,...",[75],[],0
4,4,"Услуги , оказанные без медицинских показаний ,...",[75],[],0
...,...,...,...,...,...
42178,42178,2024г . Страховая Компания _ `` Группа _ Страх...,"['8', '9']",[],1
42179,42179,07 . Страховая Компания _ `` Группа _ Страхова...,"['8', '9']",[],1
42180,42180,Страховая Компания _ `` Группа _ Страхование `...,"['8', '9']",[],1
42181,42181,Страховая Компания _ `` Группа _ Страхование `...,"['8', '9']",[],1
